# Dados Abertos - Cadastros de Empresas Brasileiras
## Arquivo Exemplo - como se utilizar das bibliotecas deste projeto

Primeiramente vamos fazer o download dos arquivos usando o bash:

In [1]:
!bash downloader.sh

--2021-11-03 09:41:40--  http://200.152.38.155/CNPJ/
Connecting to 200.152.38.155:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘200.152.38.155/index.html.tmp’

200.152.38.155/inde     [ <=>                ]   9,67K  --.-KB/s    in 0s      

2021-11-03 09:41:41 (34,2 MB/s) - ‘200.152.38.155/index.html.tmp’ saved [9906]

Loading robots.txt; please ignore errors.
--2021-11-03 09:41:41--  http://200.152.38.155/robots.txt
Reusing existing connection to 200.152.38.155:80.
HTTP request sent, awaiting response... 404 Not Found
2021-11-03 09:41:41 ERROR 404: Not Found.

Removing 200.152.38.155/index.html.tmp since it should be rejected.

--2021-11-03 09:41:41--  http://200.152.38.155/CNPJ/?C=N;O=D
Reusing existing connection to 200.152.38.155:80.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘200.152.38.155/index.html?C=N;O=D.tmp’

200.152.38.155/inde     [ <=>                ]   9,67K  --.

Descompactando os arquivos:

In [2]:
# bibliotecas
import os 
import utils

# buscando as variáveis
path = os.getcwd()
zipped = path + '/arquivos-zip/200.152.38.155/'
unzipped = path + '/arquivos-unzip/'

# iniciando o descompactador:
utils.descompactando(caminho_do_arquivo=zipped, tipo_arquivo='.zip',destino=unzipped)

arquivo descompactado:  K3241.K03200Y0.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y1.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y2.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y3.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y5.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y4.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y7.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y6.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y9.D10710.ESTABELE.zip
arquivo descompactado:  K3241.K03200Y8.D10710.ESTABELE.zip
arquivo descompactado:  F.K03200$Z.D10710.CNAECSV.zip
arquivo descompactado:  F.K03200$Z.D10710.PAISCSV.zip
arquivo descompactado:  K3241.K03200Y8.D10710.EMPRECSV.zip
arquivo descompactado:  K3241.K03200Y9.D10710.EMPRECSV.zip
arquivo descompactado:  F.K03200$Z.D10710.MOTICSV.zip
arquivo descompactado:  K3241.K03200Y9.D10710.SOCIOCSV.zip
arquivo descompactado:  K3241.K03200Y8.D10710.SOCIOCSV.zip
arquivo desc

'arquivos descompactados com sucesso'

# Como usar o leitor:

A partir dos arquivos baixados, importe a biblioteca `leitor` e chame o método desejado. A biblioteca contem como os arquivos devem ser limpos para depois poderem ser interpretados. Uma vez executada, a biblioteca irá importar os dados para a memória do seu computador usando do paralelismo do spark.

Depois de lido, os arquivos carregados poderão ser consultados usando comandos em SQL.

## Perfil dos Empresários do Brasil
Aqui temos um exemplo de como usar o leitor com os arquivos de sócios e extraindo um breve perfil por faixa etária dos sócios das empresas brasileiras.

In [5]:
# spark configs:
spark_session, sqlContext, sc = utils.spark_configs(memory='6G')

# carregando os dados
import leitor

leitor.importador_socios(arquivo='arquivos-unzip/K3241.K03200Y*.D10710.SOCIOCSV',separador=';',previsualizacao=True)

+-----------+----------------------+--------------------+--------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+---------+------------------+
|cnpj_basico|identificador_de_socio|       nome_do_socio|cnpj_cpf_socio|qualificacao_do_socio|data_de_entrada_sociedade|pais|representante_legal|nome_do_representante|qualificacao_do_representante_legal|faixa_etaria|cod_faixa| desc_faixa_etaria|
+-----------+----------------------+--------------------+--------------+---------------------+-------------------------+----+-------------------+---------------------+-----------------------------------+------------+---------+------------------+
|   17872629|                     2|KARINA FERREIRA S...|   ***945998**|                   65|                 20130404|null|        ***000000**|                 null|                                 00|           4|        4|Entre 31 a 40 anos|
|   18331805|   

'tabela de socios carregada com sucesso'

In [6]:
# empresarios por idade
query_idade = """
    SELECT 
        desc_faixa_etaria,
        COUNT(cnpj_basico) AS frequencia
    FROM socios
    GROUP BY 1 
    ORDER BY 2 DESC
    """
por_idade = sqlContext.sql(query_idade)
por_idade.show()


+------------------+----------+
| desc_faixa_etaria|frequencia|
+------------------+----------+
|Entre 41 a 50 anos|   5053330|
|Entre 51 a 60 anos|   4443532|
|Entre 31 a 40 anos|   4028059|
|Entre 61 a 70 anos|   3103257|
|Entre 71 a 80 anos|   1421454|
|Entre 21 a 30 anos|   1332746|
|Maiores de 80 anos|    682894|
|     Nao se aplica|    432114|
|Entre 13 a 20 anos|    117303|
| Entre 0 a 12 anos|     29251|
+------------------+----------+



-- Fim --